- beers per user : 373 seconds
- std : 66 seconds
- explorer score per user : 0.3 seconds per user
- adventurer score per user : 0.09 s per user

In [1]:
import pandas as pd
import time

import categories as cat

In [2]:
ratings_parquets_path = 'RefinedData/BeerAdvocate//ratings.parquet'
beers_parquet_path = "RefinedData/BeerAdvocate/beers.parquet/"
users_parquet_path = "RefinedData/BeerAdvocate/users.parquet/"

In [ ]:
t = time.time()
categorization = cat.Categorization(ratings_parquets_path, beers_parquet_path, users_parquet_path)
print("run time :",time.time()-t)

ratings loaded
beers loaded
users loaded


In [ ]:
categorization.get_ratings_head()

In [ ]:
categorization.get_users_head()

In [ ]:
categorization.get_beers_head()

In [ ]:
user_ids = categorization.get_userIds_list()
print(len(user_ids))
user_ids[:3]

In [ ]:
categorization.co

In [ ]:
adventurer_scores = categorization.get_adventurer_score()
print(adventurer_scores)
categorization.users_ddf

In [ ]:
explorer_scores = categorization.get_explorer_score_for_all_users()
print(explorer_scoresorer_scores)
categorization.users_ddf

In [ ]:
categorization.

In [ ]:
from copy import copy
import dask.dataframe as ddf
T = 3

ratings_ddf = copy(categorization.ratings_ddf)
beers_per_users = copy(categorization.beers_per_users)
users_ddf = copy(categorization.users_ddf)

adventurer_score_fast_table=categorization.ratings_ddf.loc[categorization.ratings_ddf["dated_rating"] < T].copy().groupby('uid').count()[['rank']]
adventurer_score_fast_table=adventurer_score_fast_table.reset_index()

users_ddf=ddf.merge(
                users_ddf,
                adventurer_score_fast_table,
                how="left", on='uid')
users_ddf=users_ddf.rename(columns={'rank': 'adv_score'}).fillna(0)
users_ddf['adv_score'] = users_ddf['adv_score'].astype('Int64')
users_ddf

In [ ]:
T = 3
n = 240
import numpy as np

def jupyter_get_adventurer_score(user_ids, ratings_ddf, beers_per_users):
    print("ADVENTURER SCORE FOR ", user_id)
    rating_of_user_dated = ratings_ddf.loc[ratings_ddf["uid"] == user_id][["dated_rating"]]
    rating_of_user_dated = ratings_ddf.groupby("uid").apply(list)
    
    rating_of_user_dated = rating_of_user_dated.loc[rating_of_user_dated["dated_rating"] < T]
    return len(rating_of_user_dated.index)

In [ ]:
T = 3
n = 240
import numpy as np

def jupyter_get_adventurer_score(user_id, ratings_ddf, beers_per_users):
    #print("ADVENTURER SCORE FOR ", user_id)
    #t = time.time()
    rating_of_user_dated = ratings_ddf.loc[ratings_ddf["dated_rating"] < T]
    #print("line 1", time.time()-t)

    #t = time.time()
    rating_of_user_dated = rating_of_user_dated.loc[rating_of_user_dated["uid"] == user_id][["dated_rating"]]
    #print("line 2", time.time()-t)
    return len(rating_of_user_dated.index)

In [ ]:
t = time.time()
n = 100
for user_id in user_ids[:n]:
    jupyter_get_adventurer_score(user_id, categorization.ratings_ddf, categorization.beers_per_users)
print((time.time()-t) / n)

In [ ]:
t = time.time()
jupyter_get_adventurer_score("jarodrishe.823911", categorization.ratings_ddf, categorization.beers_per_users)
print(time.time()-t)

In [ ]:
categorization.beers_ddf.loc[categorization.beers_ddf["bid"] == 17538]

In [ ]:
from copy import copy
#(ba_score+mean(avg_ratings)-mean(ba_score))/std(ba_score)4.054117*std(avf_ratings)0.385004

categorization.beers_ddf.loc[~np.isnan(categorization.beers_ddf["ba_score"])]["ba_score"]

beers_ddf = copy(categorization.beers_ddf)
beers_ddf["ba_score"] /= 20
beers_ddf

In [ ]:
def jupyter_get_explorer_score_for_user(ratings_ddf, user_id, beers_per_users):
    first_ratings = ratings_ddf.loc[ratings_ddf["rank"] <= 10][["uid"]]
    first_ratings = first_ratings.loc[first_ratings["uid"] == user_id]
    return len(first_ratings.index)

In [ ]:
from copy import copy

ratings_ddf = copy(categorization.ratings_ddf)
beers_per_users = copy(categorization.beers_per_users)

t = time.time()
n = 10
for user_id in user_ids[:n]:
    print("user id :",user_id,"score:",jupyter_get_explorer_score_for_user(ratings_ddf, user_id, beers_per_users))
print("Total time average: ", (time.time() - t)/n)

jupyter_get_explorer_score_for_user(ratings_ddf, "zymurgy4all.11427", beers_per_users)

In [ ]:
def jupyter_get_explorer_score(ratings_ddf, user_ids3, beers_per_users):
    grouped_ratings_dff = ratings_ddf.groupby("uid").apply(list)
    
    
    first_ratings = ratings_ddf.loc[ratings_ddf["rank"] <= 10][["uid"]]
    first_ratings = first_ratings.loc[first_ratings["uid"] == user_id]
    return len(first_ratings.index)

In [ ]:
from copy import copy
import dask.dataframe as ddf

ratings_ddf = copy(categorization.ratings_ddf)
beers_per_users = copy(categorization.beers_per_users)
users_ddf = copy(categorization.users_ddf)

grouped_ratings_ddf = ratings_ddf[["uid","rank"]]
grouped_ratings_ddf = grouped_ratings_ddf.loc[grouped_ratings_ddf["rank"]<=10].groupby("uid").count()

grouped_ratings_table = grouped_ratings_ddf.reset_index()

users_ddf = ddf.merge(
                users_ddf,
                grouped_ratings_table,
                how="left", on='uid')

users_ddf = users_ddf.rename(columns={'rank': 'xpl_score'}).fillna(0)
users_ddf['xpl_score'] = users_ddf['xpl_score'].astype('Int64')

print(users_ddf["xpl_score"])

users_ddf

In [ ]:
def jupyter_get_expertlike_score(user_id, beers_per_users, beers_ddf, ratings_ddf):
    beer_ids = beers_per_users[user_id]
    ref = beers_ddf.loc[beers_ddf["bid"].isin(beer_ids)][["bid","ba_score"]]
    r_user_beers = ratings_ddf.loc[ratings_ddf["bid"].isin(beer_ids)]
    std = beers_ddf["std_rating"]
    print(std)

    return


In [ ]:
from copy import copy

user_id = user_ids[0]

ratings_ddf = copy(categorization.ratings_ddf)
beers_ddf = copy(categorization.beers_ddf)
beers_per_users = copy(categorization.beers_per_users)
users_ddf = copy(categorization.users_ddf)


jupyter_get_expertlike_score(user_id, beers_per_users, beers_ddf, ratings_ddf)

In [ ]:
df =pd.DataFrame(data={'id':[111,111,111,333,444],'user_id':[11,22,33,44,55],'date': [10,1, 2,9,8], 'rating': [0,1, 1,5,6]}) 
df

In [ ]:
df.loc[df["id"] == 111][["user_id","date"]].sort_values("date")

In [ ]:
df.loc[df["id"] == 111].loc[df["rating"] == 1]

In [ ]:
type(df["col1"][0])